# utils

> Utility functions

In [ ]:
#| default_exp utils

In [ ]:
#| hide
from fastcore.test import test_eq

In [ ]:
#| export
def noop(*args, **kwargs): return None

In [ ]:
#|hide
test_eq(noop(), None)
test_eq(noop(1), None)
test_eq(noop(1,2), None)
test_eq(noop(1,2,x=3), None)
test_eq(noop((1,2)), None)
test_eq(noop({'a':1}, x=3), None)
test_eq(noop(lambda x: x), None)
test_eq(noop(noop), None)

In [ ]:
#| export
def identity(x): return x

In [ ]:
#|hide
test_eq(identity(1), 1)
test_eq(identity('a'), 'a')
test_eq(identity((1,2)), (1,2))
test_eq(identity({'a':1}), {'a':1})
test_eq(identity(noop), noop)

In [ ]:
#| export
def safe_not_equal(a,b):
    "Check if `a` is not equal to `b`"
    primitive = (int, str, bool, frozenset, tuple)
    return (a != b) if isinstance(a, primitive) else True

In [ ]:
#|hide
test_eq(safe_not_equal(1,2), True)
test_eq(safe_not_equal(0,False), False)
test_eq(safe_not_equal(object(), object()), True)

test_eq(safe_not_equal({"a":1}, {"a":1}), True)

In [ ]:
#|export
from functools import reduce
from typing import Callable, TypeVar,  Generic, Union, Optional, Set, Protocol, Any
from typing_extensions import Annotated

In [ ]:
#| export
def compose( 
    *functions # functions to be composed (left to right)
) -> Callable[[Any], Any]:  # composed function
    """Compose multiple functions left to right.\n
    Examples:\n
        compose()(x) = x 
        compose(f)(x) = f(x) 
        compose(f, g)(x) = g(f(x)) 
        ... 
    """    
    if (len(functions)==0): return lambda x: x # compose()(x) = x
    def pack(x): return x if type(x) is tuple else (x,)
    def call(f, g):
       return lambda *x: g(*pack(f(*pack(x)))) # call in reverse order
    return reduce(call, functions)  # composed function

In [ ]:
#|hide
add2 = lambda x: x+2
mul5 = lambda x: x*5
add = lambda x,y: x+y
test_eq(compose()(1), 1) # compose()(x) == x
test_eq(compose(add2)(1), add2(1)) # compose(f)(x) == f(x)
test_eq(compose(mul5)(1), mul5(1))
test_eq(compose(add)(4,2), add(4,2)) # compose(f, g)(x) == g(f(x))
test_eq(compose(add2, mul5)(1), mul5(add2(1)))
dummy = lambda a,b: f'{a}_{b}'
up = lambda a: a.upper()
test_eq(compose(dummy, up)('foo', 'bar'), up(dummy('foo', 'bar')))

In [ ]:
#|hide
#`retrieve_name` not used in the current version of the library. So, I am not exporting:
import inspect

In [ ]:
#|hide
def retrieve_name(var): # From: https://stackoverflow.com/a/40536047/1344369
        """
        Gets the name of var. Does it from the out most frame inner-wards.
        :param var: variable to get name from.
        :return: string
        """
        for fi in reversed(inspect.stack()):
            names = [var_name for var_name, var_val in fi.frame.f_locals.items() if var_val is var]
            if len(names) > 0:
                return names[0]

In [ ]:
#|hide
a = list([1,2,3])
retrieve_name(a)
test_eq(retrieve_name(a), 'a')
b = lambda x: x
test_eq(retrieve_name(b), 'b')

In [ ]:
#| export
class Bunch(dict):
    __init__     = lambda self, **kw: setattr(self, '__dict__', kw) #type: ignore
    __repr__     = lambda self: f'{self.__class__.__name__}({self.__dict__})'
    __contains__ = lambda self, k: k in self.__dict__ or hasattr(self, k)
    __bool__     = lambda self: bool(self.__dict__)
    __hash__     = lambda self: sum([hash(v) if not isinstance(v, (list, set, dict)) else len(v) for v in self.__dict__.values()]) #type: ignore
    asDict       = lambda self: self.__dict__
    asTuple      = lambda self: tuple(self.__dict__.values())

class NamedBunch(Bunch):
    def __init__(self, name, **kw):
        super().__init__(**kw)
        self.__class__.__name__ = name

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()